<a href="https://colab.research.google.com/github/aai540-group3/project/blob/main/aws_confgure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configure AWS

## Install Dependencies

In [ ]:
%%bash

# INSTALL: uv
# Rust-based Python Package Management
pip install --quiet --progress-bar=off uv

# INSTALL: boto3
# AWS SDK (Software Development Kit) for Python
uv pip install --system --quiet boto3

mkdir -p /root/.aws

## Add Credentials to Google Colab

In [ ]:
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError

try:
    aws_access_key_id = userdata.get('AWS_ACCESS_KEY_ID')
except SecretNotFoundError as e:
    print(f"Error: 'AWS_ACCESS_KEY_ID' not found.\n"
        "To set it manually in the Google Colab Secrets UI:\n"
        "1. Click the key icon on the left sidebar to open the 'Secrets' pane.\n"
        "2. Click the '+' button to add a new secret.\n"
        "3. Enter 'AWS_ACCESS_KEY_ID' as the name and your actual AWS Access Key ID as the value.\n"
        "4. Save the secret and rerun the code cell.")
print()
try:
    aws_secret_access_key = userdata.get('AWS_SECRET_ACCESS_KEY')
except SecretNotFoundError as e:
    print(f"Error: 'AWS_SECRET_ACCESS_KEY' not found.\n"
        "To set it manually in the Google Colab Secrets UI:\n"
        "1. Click the key icon on the left sidebar to open the 'Secrets' pane.\n"
        "2. Click the '+' button to add a new secret.\n"
        "3. Enter 'AWS_SECRET_ACCESS_KEY' as the name and your actual AWS Secret Access Key as the value.\n"
        "4. Save the secret and rerun the code cell.")

## Create Credential And Config Files

In [ ]:
# FILE: /root/.aws/credentials
credentials_content = f"""
[default]
aws_access_key_id = {aws_access_key_id}
aws_secret_access_key = {aws_secret_access_key}
"""

with open("/root/.aws/credentials", "w") as f:
  f.write(credentials_content)

# FILE: /root/.aws/config
config_content = f"""
[default]
region = us-west-2
"""

with open("/root/.aws/config", "w") as f:
  f.write(config_content)

## Test Credentials

In [ ]:
import boto3
import pprint

try:
    session = boto3.Session(profile_name='default')
    sts_client = session.client('sts')
    response = sts_client.get_caller_identity()
    print("Authenticated")
except Exception as e:
    print(f"Error retrieving AWS identity: {e}")

Authenticated
